In [1]:
import os.path
import cv2
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import src.metrics as nc_metrics
import src.utils as nc_utils
import notebooks.sandbox.models as nc_models

from pathlib import Path
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import models

from tensorflow.keras.metrics import Recall, Precision

2024-02-15 21:10:32.772631: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-15 21:10:32.772705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-15 21:10:32.828250: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-15 21:10:32.956968: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-15 21:10:35.706602: W tensorflow/compiler/tf2

In [ ]:
strategy = nc_utils.start_session()

ROOT_DIR = "."
IMAGE_DIR = os.path.join(ROOT_DIR, "images")
MASK_DIR = os.path.join(ROOT_DIR, "masks")

BATCH_SIZE = 8
LR = 1e-4  # Learning rate
EPOCHS = 300

smallest_dimension = nc_utils.get_smallest_image_dimension(IMAGE_DIR)

#IMAGE_SIZE = smallest_dimension
IMAGE_SIZE = 256
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
INPUT_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

In [ ]:
(x_train_paths, y_train_paths), (x_valid_paths, y_valid_paths), (
x_test_paths,
y_test_paths) = nc_utils.load_data(
    image_directory=IMAGE_DIR,
    mask_directory=MASK_DIR,
    split=0.1
)

train_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_train_paths, y_train_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

validation_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_valid_paths, y_valid_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

test_dataset = nc_utils.get_tensorflow_dataset(
    image_mask_paths=(x_test_paths, y_test_paths),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

data_aug = nc_utils.get_data_augmentation_pipeline()

In [ ]:
# Initiating model on GPU
with strategy.scope():
    model = nc_models.unet_model(input_shape=INPUT_SHAPE, augment_data=True, num_filters=8)
    metrics=[Recall(), Precision()]
    loss=nc_metrics.cdc_loss
    opt=tf.keras.optimizers.Nadam(LR)

In [ ]:
model = model()
model.summary()

In [ ]:
# Compiling model
model.compile(optimizer=opt,
              loss=loss,
              metrics=metrics)

early_stopping = callbacks.EarlyStopping(monitor='val_loss',
                                         patience=5, mode='max',
                                         restore_best_weights=True)

In [ ]:
train_steps = len(x_train_paths) // BATCH_SIZE
valid_steps = len(x_valid_paths) // BATCH_SIZE

if len(x_train_paths) % BATCH_SIZE != 0:
    train_steps += 1
if len(x_valid_paths) % BATCH_SIZE != 0:
    valid_steps += 1

try:
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=validation_dataset,
        callbacks=[early_stopping],
        steps_per_epoch=train_steps,
        validation_steps=valid_steps)
except Exception as e:
    print("An error occurred:", e)

In [ ]:
test_steps = len(x_test_paths) // BATCH_SIZE
if len(x_test_paths) % BATCH_SIZE != 0:
    test_steps += 1
    
model.evaluate(test_dataset, steps=test_steps)

# Results

In [ ]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = x/255.0
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = np.expand_dims(x, axis=-1)
    x = x/255.0
    return x

In [ ]:
def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

In [ ]:
for i, (x, y) in enumerate(zip(test_x[:10], test_y[:10])):
    x = read_image(x)
    y = read_mask(y)
    y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
    h, w, _ = x.shape
    white_line = np.ones((h, 10, 3))

    all_images = [
        x, white_line,
        mask_parse(y), white_line,
        mask_parse(y_pred)
    ]
    image = np.concatenate(all_images, axis=1)

    fig = plt.figure(figsize=(12, 12))
    a = fig.add_subplot(1, 1, 1)
    imgplot = plt.imshow(image)